# Parse de documentos HTML com a biblioteca lxml

## lxml

[Link](https://lxml.de/)

- Biblioteca (ou _toolkit_) para processar XML e HTML
- Novo _binding_ para libs em C com mesmo propósito libxml2 e libxslt, que, apesar de terem _bindings_ para python, são complexos e não pythônicos
- Surgiu como alternativa durante trabalho na Aurum para parse complexo de documentos HTML


_Binding_: interface para possibilitar que algo escrito em uma linguagem de programação seja usado com outra linguagem

## Vamos ver código!
Esta _talk_ procurou seguir, na medida do possível, a ordem da maravilhosa [documentação do lxml](https://lxml.de/4.3/lxmldoc-4.3.0.pdf), para que ela lhes pareça mais familiar, caso desejem consultá-la.

## O que vamos parsear

[Folha informativa - Violência contra as mulheres](https://www.paho.org/bra/index.php?option=com_content&view=article&id=5669:folha-informativa-violencia-contra-as-mulheres&Itemid=820) da OPAS/OMS Brasil (Organização Mundial de Saúde)

## Do que vamos precisar
A `etree` é uma árvore dos elementos do nosso HTML, que podemos iterar, editar, buscar, etc. Parseando nosso HTML com o módulo `html`, vamos criar nossa árvore.

In [122]:
import lxml.html


tree = lxml.html.parse('opas_violencia_contra_mulheres.html')

In [123]:
type(tree)

lxml.etree._ElementTree

In [124]:
lxml.html.tostring(tree)[:500]

b'<!DOCTYPE html>\n<html xmlns="https://www.w3.org/1999/xhtml" xml:lang="pt-br" lang="pt-br" dir="ltr">\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n \n\t<script type="text/javascript">\n\t\tvar pathInfo = {\n\t\t\tbase: \'templates/Responsive/\',\n\t\t\tcss: \'css/\',\n\t\t\tjs: \'js/\',\n\t\t\tswf: \'swf/\',\n\t\t}\n\t</script>\n\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t<!-- <meta name="theme-color" content="#0099d9" /> -->\n\t<meta http-equiv="content-type" content="text/html; charse'

### Temos bastante coisa que não nos interessa. O que queremos é o que tem no _body_
Primeiro, vamos pegar a raiz (_root_) da nossa árvore. A partir dela, conseguimos o _body_.

In [125]:
root = tree.getroot()
root

<Element html at 0x7f7e25b90728>

In [126]:
body = root.body
body

<Element body at 0x7f7e25b90ae8>

In [127]:
lxml.html.tostring(body)[:500]

b'<body itemscope itemtype="https://schema.org/WebPage">\n<!-- Google Tag Manager -->\n<noscript><iframe src="//www.googletagmanager.com/ns.html?id=GTM-MDCJXB" title="Google Tag Manager" style="height:0px;width:0px;display:none;visibility:hidden"><span style="visibility:hidden">Google Tag</span></iframe></noscript>\n<script type="text/javascript">(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\nnew Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\nj=d.createElement(s),dl'

### Agora que temos o body, podemos extrair algumas informações
Podemos usar métodos da `etree`, como `findall`, ou o próprio XPath. Existe também um módulo chamado `cssselect`, que precisa ser instalado separadamente.

- Com `findall`, podemos achar todos os elementos correspondentes a uma expressão de XPath simples
- Com `find`, o primeiro elemento correpondente somente é trazido
- Com `findtext`, temos no retorno já o conteúdo textual do elemento correspondente
- Com `iterfind`, nosso retorno é um `generator` de todos os elementos correspondentes

In [128]:
body.findall('.//p')[:3]

[<Element p at 0x7f7e25bc3138>,
 <Element p at 0x7f7e25bc3188>,
 <Element p at 0x7f7e25bc31d8>]

In [129]:
body.find('.//p')

<Element p at 0x7f7e25bc3138>

In [130]:
body.findtext('.//p')

'Brasil'

In [131]:
body.iterfind('.//p')

### Porém, com XPath, temos acesso a todas as _features_, e não apenas expressões simples. Então vamos usar XPath para encontrar no texto algumas estatísticas (percentuais).

In [132]:
body.xpath('//*/text()[contains(., "%")]')[:5]

['\n\t\tjQuery(window).on(\'load\', function () {\n\t\tjQuery(\'iframe[id^=twitter-widget-]\').each(function () {\n\t\tvar head = jQuery(this).contents().find(\'head\');\n\t\tif (head.length) {\n\t\thead.append(\'<style type="text/css">.timeline { max-width: 100% !important; width: 100% !important; } .timeline .stream { max-width: none !important; width: 100% !important; }</style>\');\n\t\t}\n\t\tjQuery(\'.twitter-timeline\').append(jQuery(\'\'));\n\t\t})\n\t\t});\n\t',
 'Estimativas globais publicadas pela OMS indicam que aproximadamente uma em cada três mulheres (35%) em todo o mundo sofreram violência física e/ou sexual por parte do parceiro ou de terceiros durante a vida.',
 'A maior parte dos casos é de violência infligida por parceiros. Em todo o mundo, quase um terço (30%) das mulheres que estiveram em um relacionamento relatam ter sofrido alguma forma de violência física e/ou sexual na vida por parte de seu parceiro.',
 'Globalmente, 38% dos assassinatos de mulheres são cometid

### Nosso primeiro resultado não tem nada a ver com estatísticas, é um elemento _script_. Podemos aproveitar e remover todos os scripts, para nossa árvore ficar mais limpa.

In [133]:
from lxml.html.clean import Cleaner


cleaner = Cleaner(scripts=True)
body = cleaner.clean_html(body)
body.findall('.//script')

[]

In [134]:
body.xpath('//*/text()[contains(., "%")]')[:3]

['Estimativas globais publicadas pela OMS indicam que aproximadamente uma em cada três mulheres (35%) em todo o mundo sofreram violência física e/ou sexual por parte do parceiro ou de terceiros durante a vida.',
 'A maior parte dos casos é de violência infligida por parceiros. Em todo o mundo, quase um terço (30%) das mulheres que estiveram em um relacionamento relatam ter sofrido alguma forma de violência física e/ou sexual na vida por parte de seu parceiro.',
 'Globalmente, 38% dos assassinatos de mulheres são cometidos por um parceiro masculino.']

### Suponhamos que queiramos somente estatísticas do Brasil. Na verdade nenhuma se refere somente ao Brasil, mas _suponhamos_. Se tivéssemos um documento grande, poderia ser útil eliminar esses elementos que não se referem ao Brasil

In [135]:
percentage_els = body.xpath('//*[contains(., "%")]')

for el in percentage_els:
    text = el.text_content()
    if 'Brasil' not in text and el.tag in ('li', 'p'):
        el.getparent().remove(el)
        
body.xpath('//*[contains(., "%")]')

[]

### Dissecando
- `text = el.text_content()`: pegando o texto de um elemento
- `if 'Brasil' not in text and el.tag in ('li', 'p')`: se "Brasil" não está no texto e o elemento é um item de lista ou um parágrafo
- `el.getparent().remove(el)`: pegando o elemento pai do nosso elemento e deletando nosso elemento (assim que removemos um elemento específico)

**Atenção!** Nossa lista `percentage_els` ainda contém nossos elementos. Ela é uma cópia de parte do body e fica inalterada.

In [136]:
percentage_els[-2].text_content()

'A violência por parte de parceiro e a violência sexual são perpetradas principalmente por homens contra as mulheres. O abuso sexual infantil afeta meninos e meninas. Estudos internacionais revelam que aproximadamente 20% das mulheres e 5%-10% dos homens relatam terem sido vítimas de violência sexual na infância. A violência entre os jovens, incluindo em relacionamentos, é também um grande problema. \xa0'

### Se quisermos localizar elementos por id ou classe, não precisamos usar XPath. Podemos usar os métodos `find_class` e `get_element_by_id`

In [137]:
topics = body.find_class('bkbutton')

for el in topics:
    print(el.text_content())

Principais informações 
Magnitude do problema
Fatores de risco 
Consequências para a saúde   
Impacto em crianças 
Custos sociais e econômicos  
Prevenção e resposta  
Resposta da OMS
Dia Laranja


In [138]:
body.get_element_by_id('footer')

<Element footer at 0x7f7e25bad598>

### Aproveitando que fizemos um `get_parent`, vamos ver algumas formas de "navegar" pela nossa árvore com métodos da `etree`

Pegar elementos filhos

In [139]:
body.getchildren()

[<Element noscript at 0x7f7e25b905e8>, <Element div at 0x7f7e25bc3cc8>]

Iterador dos descendentes

In [140]:
list(body.iterdescendants())[:5]

[<Element noscript at 0x7f7e25b905e8>,
 <Element span at 0x7f7e25b90818>,
 <Element div at 0x7f7e25bc3cc8>,
 <Element div at 0x7f7e25bad548>,
 <Element header at 0x7f7e25bad5e8>]

Iterador dos ascendentes

In [141]:
list(body.getchildren()[0].iterancestors())

[<Element body at 0x7f7e25bc3098>]

Pegar próximo elemento

In [142]:
body.getchildren()[0].getnext()

<Element div at 0x7f7e25bc3cc8>

Iterar sobre próximos irmãos

In [143]:
list(body.getchildren()[0].itersiblings())

[<Element div at 0x7f7e25bc3cc8>]

Pegar elemento anterior

In [144]:
body.getchildren()[1].getprevious()

<Element noscript at 0x7f7e25b905e8>

Pegar a árvore original

In [145]:
body.getroottree()

# Obrigada :)
<br>

- GitHub: alana91
- Telegram: AlanaDB
- E-mail: alanadomitbittar@pm.me
- Linkedin: https://www.linkedin.com/in/alanadomitbittar/